# DICTIONARY CREATION

In [5]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os
import math
import json

DATASET_PATH=r"C:\Users\CHINMAY\Desktop\music genre\genres"
JSON_PATH=r"C:\Users\CHINMAY\Desktop\music genre\genres\data.json"
SAMPLE_RATE=22050
DURATION=30 #seconds
SAMPLES_PER_TRACK=SAMPLE_RATE*DURATION
def save_mfcc(dataset_path,json_path,n_mfcc=13,n_fft=2048,hop_length=512,num_segments=5):

 #dictionary to store data
    data={
        "mapping":[],   #for genres
        "mfcc":[],      
        "labels":[]
    }
    
    num_samples_per_segment = int(SAMPLES_PER_TRACK/num_segments)
    expected_num_mfcc_vectors_per_segment = math.ceil(num_samples_per_segment/hop_length)
    
    #loop through all the gernres
    for i,(dirpath,dirnames,filenames) in enumerate(os.walk(dataset_path)):
        
        
        #ensure that we're not at root level
        if dirpath is not dataset_path:
            
            
            #save semantic label
            dirpath_components=dirpath.split("/")
            semantic_label=dirpath_components[-1]
            data["mapping"].append(semantic_label)
            print("\nProcessing{}".format(semantic_label))
            
            
            for f in filenames:
                
                file_path = os.path.join(dirpath,f)
                signal,sr = librosa.load(file_path,sr=SAMPLE_RATE)
                
                
                #process segments extracting mfcc and storing data
                for s in range(num_segments):
                    start_sample=num_samples_per_segment*s
                    finish_sample=start_sample+num_samples_per_segment
                    
                    mfcc=librosa.feature.mfcc(signal[start_sample:finish_sample],
                                             sr=sr,
                                             n_fft=n_fft,
                                             n_mfcc=n_mfcc,
                                             hop_length=hop_length)
                    
                    mfcc = mfcc.T
                    
                    #store mfcc for segment if it has the expected length
                    if len(mfcc) == expected_num_mfcc_vectors_per_segment:
                        data["mfcc"].append(mfcc.tolist())
                        data["labels"].append(i-1)
                        print("{}, segment:{}".format(file_path,s))
                    
    with open(json_path,"w") as fp:
        json.dump(data,fp,indent=4)
        
        
if __name__ =="__main__":
    save_mfcc(DATASET_PATH,JSON_PATH,num_segments=10)
    


ProcessingC:\Users\CHINMAY\Desktop\music genre\genres\blues
C:\Users\CHINMAY\Desktop\music genre\genres\blues\blues.00000.wav, segment:0
C:\Users\CHINMAY\Desktop\music genre\genres\blues\blues.00000.wav, segment:1
C:\Users\CHINMAY\Desktop\music genre\genres\blues\blues.00000.wav, segment:2
C:\Users\CHINMAY\Desktop\music genre\genres\blues\blues.00000.wav, segment:3
C:\Users\CHINMAY\Desktop\music genre\genres\blues\blues.00000.wav, segment:4
C:\Users\CHINMAY\Desktop\music genre\genres\blues\blues.00000.wav, segment:5
C:\Users\CHINMAY\Desktop\music genre\genres\blues\blues.00000.wav, segment:6
C:\Users\CHINMAY\Desktop\music genre\genres\blues\blues.00000.wav, segment:7
C:\Users\CHINMAY\Desktop\music genre\genres\blues\blues.00000.wav, segment:8
C:\Users\CHINMAY\Desktop\music genre\genres\blues\blues.00000.wav, segment:9
C:\Users\CHINMAY\Desktop\music genre\genres\blues\blues.00001.wav, segment:0
C:\Users\CHINMAY\Desktop\music genre\genres\blues\blues.00001.wav, segment:1
C:\Users\CHINMA

# RNN-LSTM

In [2]:
# RNN

import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras


DATA_PATH=r"C:\Users\CHINMAY\Desktop\music genre\genres\data.json"

def load_data(data_path):
    with open(data_path,"r") as fp:
        data=json.load(fp)
        
    X=np.array(data["mfcc"])
    y=np.array(data["labels"])
    return X,y

def prepare_datasets(test_size,validation_size):
    X,y = load_data(DATA_PATH)
    
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=test_size)
    
    X_train,X_validation,y_train,y_validation = train_test_split(X_train,y_train,test_size=validation_size)
    
    
    #3d array ->(130,13,1)
    #X_train = X_train[...,np.newaxis]  #4d array ->(num_samples,130,13,1)
    #X_validation = X_validation[...,np.newaxis]
    #X_test = X_test[...,np.newaxis]
    
    return X_train,X_validation,X_test,y_train,y_validation,y_test

def build_model(input_shape):
    
    #create RNN-LSTM model
    model=keras.Sequential()
    # 2 LSTM layers
    model.add(keras.layers.LSTM(64,input_shape=input_shape,return_sequences=True))
    model.add(keras.layers.LSTM(64))
    
    #dense layer
    model.add(keras.layers.Dense(64,activation = 'relu'))
    model.add(keras.layers.Dropout(0.3))
    
    
    
    model.add(keras.layers.Dense(10,activation='softmax'))
    
    return model
  
def predict(model,X,y):
    
    X =X[np.newaxis,...]
    
    prediction=model.predict(X)
    
    #extract index with max value
    
    predicted_index = np.argmax(prediction,axis = 1)
    print("expected index :{},Predicted index :{}".format(y,predicted_index))
    
if __name__=="__main__":
    #create train and validation and test sets
    X_train,X_validation,X_test,y_train,y_validation,y_test = prepare_datasets(0.25,0.2)
    
    #build the cnn net
    input_shape= (X_train.shape[1],X_train.shape[2])
    
    model= build_model(input_shape)
    
    #compile
    optimizer=keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer,
                 loss="sparse_categorical_crossentropy",
                metrics=['accuracy'])
    
    #train
    model.fit(X_train,y_train,validation_data=(X_validation,y_validation),batch_size = 32,epochs =30)
    
    
    #evaluate on test
    test_error,test_accuracy = model.evaluate(X_test , y_test,verbose = 1)
    print("Accuracy on test set is :{}".format(test_accuracy))
    
    
    
    
    
    

Train on 5997 samples, validate on 1500 samples
Epoch 1/30
5997/5997 [==============================] - 19s 3ms/sample - loss: 2.2669 - accuracy: 0.1502 - val_loss: 2.1829 - val_accuracy: 0.2687
Epoch 2/30
5997/5997 [==============================] - 16s 3ms/sample - loss: 2.0785 - accuracy: 0.2855 - val_loss: 1.8904 - val_accuracy: 0.3760
Epoch 3/30
5997/5997 [==============================] - 21s 4ms/sample - loss: 1.7939 - accuracy: 0.3835 - val_loss: 1.6383 - val_accuracy: 0.4367
Epoch 4/30
5997/5997 [==============================] - 18s 3ms/sample - loss: 1.6393 - accuracy: 0.4319 - val_loss: 1.5171 - val_accuracy: 0.4753
Epoch 5/30
5997/5997 [==============================] - 21s 3ms/sample - loss: 1.5564 - accuracy: 0.4534 - val_loss: 1.4639 - val_accuracy: 0.4773
Epoch 6/30
5997/5997 [==============================] - 21s 4ms/sample - loss: 1.4762 - accuracy: 0.4854 - val_loss: 1.4071 - val_accuracy: 0.4973
Epoch 7/30
5997/5997 [==============================] - 20s 3ms/sample

# PREDICT

In [6]:
X = X_test[10]
y=y_test[10]
predict(model,X,y)
    

expected index :7,Predicted index :[7]
